In [1]:
from dotenv import load_dotenv 

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq 

llm = ChatGroq(model="openai/gpt-oss-20b")

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

msg = llm.invoke([
    SystemMessage(content="You are a helpful AI bot that assists a user in choosing the perfect book to read in one sentence"),
    HumanMessage(content="I enjoy mystery novels, what should I read?")
])

print(msg) 

content='Try Dashiell\u202fHammett’s *The Maltese Falcon*—a gritty, atmospheric mystery that’s a cornerstone of the genre and will keep you guessing until the very last page.' additional_kwargs={'reasoning_content': 'We need to respond in one sentence. The user: "I enjoy mystery novels, what should I read?" We must give a recommendation. Should we ask clarifying? No, the instruction: "You are a helpful AI bot that assists a user in choosing the perfect book to read in one sentence". So we need to give a recommendation in one sentence. Probably mention a popular mystery novel. Could mention "The Girl with the Dragon Tattoo" or "Gone Girl" or "The Hound of the Baskervilles". But maybe pick something classic like "The Maltese Falcon" or "The Big Sleep". We can also suggest something like "The Silent Patient" but that\'s more psychological thriller. The user didn\'t specify subgenre. Maybe give a recommendation that covers intrigue. I should mention one book and maybe a brief reason. But k

In [4]:
from langchain_core.messages import AIMessage

msg = llm.invoke([
    SystemMessage(content="You are a supportive AI bot that suggests fitness activities to a user in a short sentence"),
    HumanMessage(content="I like high intensity workouts, what should I do?"),
    AIMessage(content="You should try a crossfit class"),
    HumanMessage(content="How often should I attend this class?")
])

print(msg)

content='Aim for 2–3 CrossFit sessions per week, leaving at least one rest or lighter‑day in between.' additional_kwargs={'reasoning_content': 'The user: "How often should I attend this class?" The user wants a recommendation. The system says supportive AI bot suggests fitness activities. We can give a short sentence: "Aim for 2–3 sessions per week, spaced by rest days." The user asked "How often should I attend this class?" So answer: 2-3 times per week. Should we mention rest? It\'s supportive. So a short sentence.'} response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 127, 'total_tokens': 248, 'completion_time': 0.12206148, 'completion_tokens_details': {'reasoning_tokens': 88}, 'prompt_time': 0.007180719, 'prompt_tokens_details': None, 'queue_time': 0.043175631, 'total_time': 0.129242199}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'

In [5]:
from langchain_core.prompts import PromptTemplate

# when we use the PromptTemplate.from_template lc automatically detects the input_variables
# if we use PromptTemplate base class we need to specify separately

prompt = PromptTemplate.from_template("Tell me {adjective} joke about {topic}")
input_ = {"adjective":"funny","topic":"Spider-Man"}
prompt.invoke(input_) 

StringPromptValue(text='Tell me funny joke about Spider-Man')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("user","Tell me a joke about this {topic}")
])

input_ = {"topic":"Spider-Man"}
prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about this Spider-Man', additional_kwargs={}, response_metadata={})])

In [7]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

input_ = {"msgs":[HumanMessage(content="What is the day after Tuesday?")]}
prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the day after Tuesday?', additional_kwargs={}, response_metadata={})])

In [8]:
chain = prompt | llm
response = chain.invoke(input_)
response 

AIMessage(content='The day after Tuesday is **Wednesday**.', additional_kwargs={'reasoning_content': 'User asks: "What is the day after Tuesday?" It\'s a simple question: Wednesday. We answer: Wednesday.'}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 86, 'total_tokens': 128, 'completion_time': 0.042387637, 'completion_tokens_details': {'reasoning_tokens': 24}, 'prompt_time': 0.004136588, 'prompt_tokens_details': None, 'queue_time': 0.043538618, 'total_time': 0.046524225}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_deb540145b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019ca38f-0434-7e22-90bb-bc8e731acf8e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 86, 'output_tokens': 42, 'total_tokens': 128, 'output_token_details': {'reasoning': 24}})

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel,Field 

class Joke(BaseModel):
    setup: str = Field(description="question to setup a joke")
    punchline: str = Field(description="answer to resolve the joke")

joke_query = "Tell me joke."

output_parser = JsonOutputParser(pydantic_object=Joke)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="Answer the query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":format_instructions}
)

chain = prompt | llm | output_parser
response = chain.invoke({"query":joke_query})
response

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything.'}

In [10]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="Answer the user query. {format_instructions}\nList five {subject}.",
    input_variables=['subject'],
    partial_variables={"format_instructions":format_instructions}
)

chain = prompt | llm | output_parser

response = chain.invoke({"subject":"best chicken biryanis"})
response

['Hyderabadi Chicken Biryani',
 'Lucknowi Chicken Biryani',
 'Kolkata Chicken Biryani',
 'Bhopali Chicken Biryani',
 'Malabar Chicken Biryani']

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import Field,BaseModel

class Movie(BaseModel):
    title: str = Field(description="Movie title")
    director: str = Field(description="Movie directors name")
    year: int = Field(description="Movie release year")
    genre: str = Field(description="Movie genre")


output_parser = JsonOutputParser()
format_instructions = """RESPONSE FORMAT: Return ONLY a single JSON object-no markdown, no example, no extra keys. It must look exactly like:
{
  "title":"movie title",
  "director":"director name",
  "year":"release year",
  "genre":"movie genre"
}

IMPORTANT: Your response must be *only* JSON. Do NOT include any illustrate or example JSON."""

prompt_template = PromptTemplate(
    template="""You are a JSON only assistant.
    
    Task: Generate info about the {movie_name} in JSON format.

    {format_instructions}
    """,
    input_variables=["movie_name"],
    partial_variables={"format_instructions":format_instructions}
)

movie_chain = prompt_template | llm | output_parser

movie_name = "Spider-Man:Into The Spider-Verse"
result = movie_chain.invoke({"movie_name":movie_name})
result

{'title': 'Spider-Man: Into the Spider-Verse',
 'director': 'Bob Persichetti, Peter Ramsey, Rodney Rothman',
 'year': '2018',
 'genre': 'Animation, Action, Adventure, Superhero'}

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf")
document = loader.load()

In [13]:
print(document[0].page_content)

* corresponding author - jkim72@kent.edu 
Revolutionizing Mental Health Care through 
LangChain: A Journey with a Large Language 
Model
Aditi Singh 
 Computer Science  
 Cleveland State University  
 a.singh22@csuohio.edu 
Abul Ehtesham  
The Davey Tree Expert 
Company  
abul.ehtesham@davey.com 
Saifuddin Mahmud  
Computer Science & 
Information Systems  
 Bradley University  
smahmud@bradley.edu  
Jong-Hoon Kim* 
 Computer Science,  
Kent State University,  
jkim72@kent.edu 
Abstract— Mental health challenges are on the rise in our 
modern society, and the imperative to address mental disorders, 
especially regarding anxiety, depression, and suicidal thoughts, 
underscores the need for effective interventions. This paper 
delves into the application of recent advancements in pretrained 
contextualized language models to introduce MindGuide, an 
innovative chatbot serving as a mental health assistant for 
individuals seeking guidance and support in these critical areas. 
MindGuide leve

In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
web_data = loader.load()
print(web_data[0].page_content[:1000])

USER_AGENT environment variable not set, consider setting it to identify your requests.


LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageOpen sourceSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewDeep AgentsLangChainLangGraphIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewPrebuilt middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this page Create an agent Core benefitsLangChain overviewCopy pageLangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolvesCopy pageLangChain is the easy way to start building completely custom agents a

In [15]:
from langchain_text_splitters import CharacterTextSplitter

text_splitters = CharacterTextSplitter(chunk_size=200,chunk_overlap=20,separator="\n")
chunks = text_splitters.split_documents(document)
len(chunks)

147

In [16]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader,WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter,RecursiveCharacterTextSplitter

paper_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf"
pdf_loader = PyPDFLoader(paper_url)
pdf_document = pdf_loader.load()

web_url =  "https://docs.langchain.com/"

web_loader = WebBaseLoader(web_url)
web_document = web_loader.load()

splitter_1 = CharacterTextSplitter(chunk_size=300,chunk_overlap=30,separator="\n")
splitter_2 = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50,separators=["\n\n","\n"," ",""])

chunks_1 = splitter_1.split_documents(pdf_document)
chunks_2 = splitter_2.split_documents(pdf_document)

def display_document_stats(docs, name):
    """Display statistics about a list of document chunks"""
    total_chunks = len(docs)
    total_chars = sum(len(doc.page_content) for doc in docs)
    avg_chunk_size = total_chars / total_chunks if total_chunks > 0 else 0
    
    # Count unique metadata keys across all documents
    all_metadata_keys = set()
    for doc in docs:
        all_metadata_keys.update(doc.metadata.keys())
    
    # Print the statistics
    print(f"\n=== {name} Statistics ===")
    print(f"Total number of chunks: {total_chunks}")
    print(f"Average chunk size: {avg_chunk_size:.2f} characters")
    print(f"Metadata keys preserved: {', '.join(all_metadata_keys)}")
    
    if docs:
        print("\nExample chunk:")
        example_doc = docs[min(5, total_chunks-1)]  # Get the 5th chunk or the last one if fewer
        print(f"Content (first 150 chars): {example_doc.page_content[:150]}...")
        print(f"Metadata: {example_doc.metadata}")
        
        # Calculate length distribution
        lengths = [len(doc.page_content) for doc in docs]
        min_len = min(lengths)
        max_len = max(lengths)
        print(f"Min chunk size: {min_len} characters")
        print(f"Max chunk size: {max_len} characters")

# Display stats for both chunk sets
display_document_stats(chunks_1, "Splitter 1")
display_document_stats(chunks_2, "Splitter 2")



=== Splitter 1 Statistics ===
Total number of chunks: 95
Average chunk size: 263.80 characters
Metadata keys preserved: creator, total_pages, page_label, author, source, moddate, creationdate, page, title, producer

Example chunk:
Content (first 150 chars): comprehensive support within the field of mental health. 
Additionally, the paper discusses the implementation of 
Streamlit to enhance the user ex pe...
Metadata: {'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}
Min chunk size: 49 characters
Max chunk size: 299 characters

=== Splitter 2 Statistics ===
Total number of chunks: 57
Average chunk size: 452.74 characters
Metadata keys preserved: creator, total_pages, page_label, author, sour

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma

paper_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf"
pdf_loader = PyPDFLoader(paper_url)
docs = pdf_loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks = splitter.split_documents(docs)
print("chunks:",len(chunks))

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(documents=chunks,
                                     embedding=embedding_model,
                                     persist_directory="./chroma_db")

print("Chromadb created")


chunks: 57
Chromadb created


In [18]:
query = "What is LangChain is used for?"
docs = vector_store.similarity_search(query,k=3)

for i,doc in enumerate(docs):
    print(f"\nResults:{i + 1}")
    print(doc.page_content[:200])


Results:1
LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether you

Results:2
LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether you

Results:3
LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether you


In [19]:
texts = [doc.page_content for doc in chunks]
embeddings = embedding_model.embed_documents(texts)
print(len(embeddings))
print(len(embeddings[0]))

57
384


In [20]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000019A8CA3AF60>, search_kwargs={})

In [21]:
docs = retriever.invoke("LangChain")
docs[0]

Document(id='870b2de9-df4e-4655-b936-0f6b0935dd1c', metadata={'total_pages': 6, 'title': 's8329 final', 'creator': 'Microsoft Word', 'author': 'IEEE', 'creationdate': '2023-12-31T03:50:13+00:00', 'page': 0, 'page_label': '1', 'producer': 'PyPDF', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'moddate': '2023-12-31T03:52:06+00:00'}, page_content='and human. The conclusion is drawn in Section V. \nII. LANGCHAIN \nLangChain, with its open -source essence, emerges as a \npromising solution, aiming to simplify the complex process of \ndeveloping applications powered by large language models \n(LLMs). This framework though the rapid delivery of building \nblocks and pre-built chains for building large language model \napplications shows the easy way developers can do it.')

In [22]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf")
docs = loader.load()

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100,separators=["\n\n","\n"," ",""])
splits = splitter.split_documents(docs)

In [24]:
from langchain_chroma import Chroma 
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(documents=splits,
                                     embedding=embeddings,
                                     collection_name="pdf_rag")

retriever = vector_store.as_retriever()

In [25]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-20b")

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter

prompt = ChatPromptTemplate.from_template(
    template=
    """
    You are a helpful assistant. 
    Answer the question ONLY using the context below.

    Context:
    {context}

    Question:
    {question}
    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context":itemgetter("question") | retriever | format_docs,
        "question":itemgetter("question")
    }
    | prompt 
    | llm
    | StrOutputParser()
)
 


In [27]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# RunnableWithMessageHistory: wrapper that adds memory to the langchain agent
# InMemoryChatMessageHistory: memory storage class that stores chat messages in RAM 

store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chat_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

chat_chain.invoke({"question":"What is the paper about?"},
                  config={"configurable":{"session_id":"1"}})


'The paper examines the urgent link between mental health problems and suicide risk, highlighting how many people—especially young adults—die or attempt suicide each year. It argues that early identification of suicidal ideation is a practical way to reduce these deaths. The study reviews recent advances in deep‑learning and natural‑language‑processing techniques, focusing on how contextualized, pretrained language models can be leveraged to detect warning signs of suicidal thoughts in textual data. In short, the paper proposes that cutting‑edge NLP and AI methods can help clinicians spot at‑risk individuals sooner and intervene more effectively.'

In [28]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                               chunk_overlap=20,
                                               separators=["\n\n","\n"," ",""])

chunks = text_splitter.split_documents(docs)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(documents=chunks,
                                     embedding=embedding_model,
                                     collection_name="simple-rag")

retriever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant. 
    Answer ONLY using the context below. 

    Context:
    {context}

    Question:
    {question}
    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context":itemgetter("question") | retriever | format_docs,
        "question":itemgetter("question")
    }
    | prompt
    | llm 
    | StrOutputParser()
)


test_queries = [
    "What is LangChain?",
    "How do retrievers work?",
    "Why is document splitting important?"
]

for q in test_queries:
    print("Q:", q)
    print("A:", rag_chain.invoke({"question": q}))
    print("-" * 50)


Q: What is LangChain?
A: LangChain is a framework that lets you build applications powered by large language models (LLMs). It provides a pre‑built agent architecture built on top of LangGraph, enabling features such as durable execution, streaming, human‑in‑the‑loop interactions, and persistence. The framework also supplies easy model integrations so you can quickly incorporate LLMs into your agents and applications.
--------------------------------------------------
Q: How do retrievers work?
A: The provided context does not include details about how retrievers work.
--------------------------------------------------
Q: Why is document splitting important?
A: The context does not mention document splitting.
--------------------------------------------------


In [29]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-20b")

history = InMemoryChatMessageHistory()
history.add_ai_message("hello!")

history.add_user_message("What is the capital of Hyderabad?")

# print(history.messages)
response = llm.invoke(history.messages)
print(response.content)

history.add_ai_message(response.content)

Hyderabad itself is a city, so it doesn’t have a “capital” in the way a country or state does. However, Hyderabad is the capital city of the Indian state of **Telangana** (and it was also the capital of the former state of Andhra Pradesh until 2014).


In [30]:
history.messages

[AIMessage(content='hello!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='What is the capital of Hyderabad?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hyderabad itself is a city, so it doesn’t have a “capital” in the way a country or state does. However, Hyderabad is the capital city of the Indian state of **Telangana** (and it was also the capital of the former state of Andhra\u202fPradesh until 2014).', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [31]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    MessagesPlaceholder("history"),
    ("human","{input}")
])

In [32]:
chain = prompt | llm

In [33]:
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [34]:
from langchain_core.runnables import RunnableWithMessageHistory

chat_chain = RunnableWithMessageHistory(chain,
                                        get_session_history,
                                        input_messages_key="input",
                                        history_messages_key="history")

In [35]:
# conversation 
chat_chain.invoke(
    {"input": "Hello, I am a Siberian Tiger. Who are you?"},
    config={"configurable": {"session_id": "1"}}
)

chat_chain.invoke(
    {"input": "What can you do?"},
    config={"configurable": {"session_id": "1"}}
)

chat_chain.invoke(
    {"input": "Who am I?"},
    config={"configurable": {"session_id": "1"}}
)


AIMessage(content='You’re a Siberian tiger—one of the world’s largest and most iconic big‑cat species, native to the forests and grasslands of the Russian Far East. Known for its powerful build, striking golden coat with black stripes, and majestic presence, the Siberian tiger is a symbol of strength and natural beauty.', additional_kwargs={'reasoning_content': 'The user says: "Who am I?" They previously said "Hello, I am a Siberian Tiger. Who are you?" They responded "Hello, I\'m ChatGPT". Now they ask "Who am I?" They might be expecting the assistant to answer that they are a Siberian tiger, as per the earlier statement. But we need to handle the context: The user claims to be a Siberian tiger. So the assistant can confirm that they are a Siberian tiger. But we should also be mindful of the policy: The assistant should not be defaming or providing disallowed content. No disallowed content. It\'s fine. We can respond: "You are a Siberian tiger." But perhaps we can add a bit of context

In [36]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,AIMessage

llm = ChatGroq(model="openai/gpt-oss-20b",
               max_tokens=256,
               temperature=0.2)

history = InMemoryChatMessageHistory()

history.add_user_message("Hello, my name is Alice.")
history.add_ai_message("Hello, Alice.")
print(history.messages)

[HumanMessage(content='Hello, my name is Alice.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Alice.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant."),
    MessagesPlaceholder("history"),
    ("human","{input}")
])

chain = prompt | llm | StrOutputParser()

store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chat_chain = RunnableWithMessageHistory(chain,
                                        get_session_history,
                                        input_messages_key="input",
                                        history_messages_key="history")

def chat_simulation(conversation, inputs, session_id="1"):
    print("\n=== Beginning Chat Simulation ===")
    
    for i, user_input in enumerate(inputs):
        print(f"\n--- Turn {i+1} ---")
        print(f"Human: {user_input}")
        
        response = conversation.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )
        
        print(f"AI: {response}")
    
    print("\n=== End of Chat Simulation ===")

# 6. Test with a series of related questions
test_inputs = [
    "My favorite color is blue.",
    "I enjoy hiking in the mountains.",
    "What activities would you recommend for me?",
    "What was my favorite color again?",
    "Can you remember both my name and my favorite color?"
]

chat_simulation(chat_chain, test_inputs,session_id="1")



=== Beginning Chat Simulation ===

--- Turn 1 ---
Human: My favorite color is blue.
AI: That’s a classic choice! Blue often feels calm and refreshing. Do you have a particular shade you love most—like navy, sky blue, or something else?

--- Turn 2 ---
Human: I enjoy hiking in the mountains.
AI: That’s awesome! The mountains have such a unique mix of challenge and serenity. Do you have a favorite range or trail you’ve tackled recently? And if you’re looking for new routes or gear tips, I’d love to help out!

--- Turn 3 ---
Human: What activities would you recommend for me?
AI: Here are a few ideas that blend your love of mountain hiking with fresh, blue‑tinged adventures:

| Activity | Why it fits | Quick Tips |
|----------|-------------|------------|
| **Mountain Biking** | Combines the trail‑loving vibe with a bit of speed. Many trails wind past alpine lakes that sparkle blue. | Start with a beginner trail, bring a helmet, and pack a small first‑aid kit. |
| **Rock Climbing / Boulder

In [38]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pprint import pprint

output_parser = StrOutputParser()

location_template = """Your job is to come up with a classic dish from the area that suggests {location}

YOUR RESPONSE:
"""

dish_template = """Given a meal {meal}, give a short and simple recipe on how to make that dish at home.

YOUR RESPONSE:
"""

time_template = """Given the recipe {recipe}, estimate how much time I need to cook it.

YOUR RESPONSE:
"""

location_chain_lcel = (
    PromptTemplate.from_template(location_template)
    | llm 
    | output_parser
) 

dish_chain_lcel = (
    PromptTemplate.from_template(dish_template)
    | llm
    | output_parser
)

time_chain_lcel = (
    PromptTemplate.from_template(time_template)
    | llm
    | output_parser
)


# Combine all chains into a single workflow using RunnablePassthrough.assign
# RunnablePassthrough.assign adds new keys to the input dictionary without removing existing ones

overall_chain_lcel = (
    # Step 1: Generate a meal based on location and add it to the input dictionary
    RunnablePassthrough.assign(meal=lambda x: location_chain_lcel.invoke({"location": x["location"]}))
    # Step 2: Generate a recipe based on the meal and add it to the input dictionary
    | RunnablePassthrough.assign(recipe=lambda x: dish_chain_lcel.invoke({"meal": x["meal"]}))
    # Step 3: Estimate cooking time based on the recipe and add it to the input dictionary
    | RunnablePassthrough.assign(time=lambda x: time_chain_lcel.invoke({"recipe": x["recipe"]}))
)

result = overall_chain_lcel.invoke({"location": "China"})
pprint(result)


{'location': 'China',
 'meal': '**Peking Duck** – a celebrated classic from Beijing, known for its '
         'crispy skin, tender meat, and the tradition of serving it with thin '
         'pancakes, scallions, cucumber, and sweet bean sauce.',
 'recipe': '**Home‑Style Peking Duck (Quick & Easy)**  \n'
           '*(Makes 2–3 servings)*  \n'
           '\n'
           '---\n'
           '\n'
           '### Ingredients  \n'
           '- 1 whole duck (≈3–4\u202flb), cleaned and patted dry  \n'
           '- 1 tbsp honey (or 2 tbsp sugar)  \n'
           '- 1 tbsp maltose or corn syrup (optional, for extra crispness)  \n'
           '- 1 tsp salt  \n'
           '- 1 tsp Chinese five‑spice powder  \n'
           '- 1 tsp ground ginger  \n'
           '- 1 tsp ground garlic  \n'
           '- 1 tbsp soy sauce  \n'
           '- 1 tbsp Shaoxing wine (or dry sherry)  \n'
           '- 1 tbsp rice vinegar  \n'
           '- 1 tsp sesame oil  \n'
           '- 1 cup water (for the glaze)  \

In [39]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Sample product reviews for testing
positive_review = """I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in grinder saves me so much time in the morning, and the programmable timer means 
I wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast."""

negative_review = """Disappointed with this laptop. It's constantly overheating after just 30 minutes of use, 
and the battery life is nowhere near the 8 hours advertised - I barely get 3 hours. 
The keyboard has already started sticking on several keys after just two weeks. Would not recommend to anyone."""

# Step 1: Define the prompt templates for each processing step
sentiment_template = """Analyze the sentiment of the following product review as positive, negative, or neutral.
Provide your analysis in the format: "SENTIMENT: [positive/negative/neutral]"

Review: {review}

Your analysis:
"""

summary_template = """Summarize the following product review into 3-5 key bullet points.
Each bullet point should be concise and capture an important aspect mentioned in the review.

Review: {review}
Sentiment: {sentiment}

Key points:
"""

response_template = """Write a helpful response to a customer based on their product review.
If the sentiment is positive, thank them for their feedback. If negative, express understanding 
and suggest a solution or next steps. Personalize based on the specific points they mentioned.

Review: {review}
Sentiment: {sentiment}
Key points: {summary}

Response to customer:
"""

sentiment_prompt = PromptTemplate.from_template(sentiment_template)
summary_prompt = PromptTemplate.from_template(summary_template)
response_prompt = PromptTemplate.from_template(response_template)

sentiment_chain = (
    sentiment_prompt
    | llm 
    | StrOutputParser()
)

summary_chain = (
    summary_prompt
    | llm 
    | StrOutputParser()
)

response_chain = (
    response_prompt
    | llm
    | StrOutputParser()
)

lcel_chain = (
    RunnablePassthrough.assign(sentiment = lambda x: sentiment_chain.invoke({"review":x["review"]}))
    | RunnablePassthrough.assign(summary = lambda x: summary_chain.invoke({"review":x["review"],
                                                                           "sentiment":x["sentiment"]}))
    | RunnablePassthrough.assign(response = lambda x: response_chain.invoke({"review":x["review"],
                                                                             "sentiment":x["sentiment"],
                                                                             "summary":x["summary"]}))
)

def test_chains(review):
    """Test both chain implementations with the given review"""
    print("\n" + "="*50)
    print(f"TESTING WITH REVIEW:\n{review[:100]}...\n")

    
    print("\nLCEL CHAIN RESULTS:")
    result = lcel_chain.invoke({"review":review})
    
    print("="*50)
    print(result)

# Run tests
test_chains(positive_review)
test_chains(negative_review)



TESTING WITH REVIEW:
I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in g...


LCEL CHAIN RESULTS:
{'review': 'I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. \nThe built-in grinder saves me so much time in the morning, and the programmable timer means \nI wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast.', 'sentiment': 'SENTIMENT: positive', 'summary': '- Brews quickly with excellent taste.  \n- Built‑in grinder saves morning prep time.  \n- Programmable timer delivers fresh coffee at wake‑up.  \n- Worth the price and highly recommended for coffee lovers.', 'response': 'Hi there!\n\nThank you so much for taking the time to share your experience with our coffee maker. We’re thrilled to hear that it’s become a morning lifesaver for you—especially the quick brew, the built‑in grinder, and the programmable timer that delivers fresh coffee right when you

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Sample product reviews for testing
positive_review = """I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in grinder saves me so much time in the morning, and the programmable timer means 
I wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast."""

negative_review = """Disappointed with this laptop. It's constantly overheating after just 30 minutes of use, 
and the battery life is nowhere near the 8 hours advertised - I barely get 3 hours. 
The keyboard has already started sticking on several keys after just two weeks. Would not recommend to anyone."""

# Step 1: Define the prompt templates for each processing step
sentiment_template = """Analyze the sentiment of the following product review as positive, negative, or neutral.
Provide your analysis in the format: "SENTIMENT: [positive/negative/neutral]"

Review: {review}

Your analysis:
"""

summary_template = """Summarize the following product review into 3-5 key bullet points.
Each bullet point should be concise and capture an important aspect mentioned in the review.

Review: {review}
Sentiment: {sentiment}

Key points:
"""

response_template = """Write a helpful response to a customer based on their product review.
If the sentiment is positive, thank them for their feedback. If negative, express understanding 
and suggest a solution or next steps. Personalize based on the specific points they mentioned.

Review: {review}
Sentiment: {sentiment}
Key points: {summary}

Response to customer:
"""

sentiment_prompt = PromptTemplate.from_template(template=sentiment_template)
summary_prompt = PromptTemplate.from_template(template=summary_template)
response_prompt = PromptTemplate.from_template(template=response_template)

senitment_chain = (
    sentiment_prompt
    | llm 
    | StrOutputParser()
)

summary_chain = (
    summary_prompt
    | llm 
    | StrOutputParser()
)

response_chain = (
    response_prompt
    | llm 
    | StrOutputParser()
)

# RunnablePassthrough is designed to accept keyword arguments, 
# where each key becomes a new field added to the input dict
overall_chain = (
    RunnablePassthrough(sentiment=lambda x:sentiment_chain.invoke({"review":x["review"]}))
    | RunnablePassthrough(summary=lambda x:summary_chain.invoke({"review":x["review"],
                                                                 "summary":x["summary"]}))
    | RunnablePassthrough(response=lambda x:response_chain.invoke({"review":x["review"],
                                                                   "summary":x["summary"],
                                                                   "response":x["response"]}))
)

def test_chains(review):
    """Test both chain implementations with the given review"""
    print("\n" + "="*50)
    print(f"TESTING WITH REVIEW:\n{review[:100]}...\n")

    
    print("\nLCEL CHAIN RESULTS:")
    result = lcel_chain.invoke({"review":review})
    
    print("="*50)
    print(result)

# Run tests
test_chains(positive_review)
test_chains(negative_review)



TESTING WITH REVIEW:
I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in g...


LCEL CHAIN RESULTS:
{'review': 'I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. \nThe built-in grinder saves me so much time in the morning, and the programmable timer means \nI wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast.', 'sentiment': 'SENTIMENT: positive', 'summary': '- Brews quickly, delivering a consistently great-tasting cup.  \n- Built‑in grinder cuts morning prep time significantly.  \n- Programmable timer ensures fresh coffee right when you wake up.  \n- Overall value: worth the price and highly recommended for coffee lovers.', 'response': 'Hi there!  \n\nThank you so much for taking the time to share your experience. We’re thrilled to hear that our coffee maker is becoming a morning essential for you—especially the quick brew, the built‑in grinder that save

### Agents 

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="openai/gpt-oss-20b",
                        model_provider="groq")


In [45]:
from langchain.tools import tool

@tool 
def calculator(eqn: str):
    """Solves the math equation and returns result"""
    return eval(eqn)



In [46]:
from langchain.agents import create_agent 

agent = create_agent(model=model,
                     tools=[calculator],
                     system_prompt="You are a math expert. You help solve any equation with the tools available")


In [48]:
response = agent.invoke({"messages":"Solve 23*90**2 + 1092 - 129 ="})

In [49]:
response['messages'][-1].content

'The value of the expression is  \n\n\\[\n23 \\times 90^{2} + 1092 - 129 = 187\\,263\n\\]'